In [1]:
from langchain_groq import ChatGroq


In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.ibm.com/job/21035348/computer-science-ai-reasoning-foundations-intern-2025-yorktown-heights-ny/?codes=SN_LinkedIn&Codes=SN_LinkedIn")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.




















Computer Science AI Reasoning Foundations Intern: 2025 - US | IBM
















































 



Skip to content



Toggle NavigationIBM CareersEnglish简体中文FrançaisFrançais Canadien日本語PortuguêsEspañolDeutschIBM Careers IBM Careers      English    简体中文    Français    Français Canadien    日本語    Português    Español    Deutsch      
 





Job Details2024-01-04T09:25:13+00:00 

727371BR


Computer Science AI Reasoning Foundations Intern: 2025    Multiple Locations See All  Yorktown Heights  San Jose  Cambridge  Albany    Research Intern   Apply now Computer Science AI Reasoning Foundations Intern: 2025    Multiple Locations See All  Yorktown Heights  San Jose  Cambridge  Albany    Research Intern   Apply now IntroductionThe Exploratory Computer Sciences program at IBM Research is seeking candidates who will contribute to foundational research topics in AI. Start and end dates for this Internship are during Summer 2025 (3 months) at IBM Research in Y

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Computer Science AI Reasoning Foundations Intern: 2025',
 'experience': 'Graduate students pursuing PhD or MS',
 'skills': ['AI/ML/Math/CS',
  'AI architectures',
  'foundation models',
  'reasoning',
  'Python',
  'PyTorch',
  'TensorFlow',
  'AI reasoning',
  'symbolic and neuro-symbolic AI methods'],
 'description': 'The Exploratory Computer Sciences program at IBM Research is seeking candidates who will contribute to foundational research topics in AI. This is for a 2025 summer internship with the following start dates: May – August or June – September for quarter system schools. IBM research interns are expected to develop innovative ideas and concepts (theoretical or experimental) while performing advanced scientific or engineering tasks resulting in highly complex, original and creative scientific or engineering achievements.'}

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd

df = pd.read_csv("ai_data_roles_portfolios.csv")
df

,Tech Stack,Links
0,"Python Developer (Backend, API)",https://example.com/python-developer-portfolio
1,"Machine Learning Engineer (Python, TensorFlow,...",https://example.com/ml-engineer-portfolio
2,"AI Engineer (Python, TensorFlow, OpenCV)",https://example.com/ai-engineer-portfolio
3,"MLOps Engineer (Kubeflow, Docker, Kubernetes)",https://example.com/mlops-portfolio
4,"Data Analyst (Python, SQL, Power BI)",https://example.com/data-analyst-portfolio
5,"Data Engineer (Python, Spark, AWS)",https://example.com/data-engineer-portfolio
6,"Data Scientist (Python, scikit-learn, Pandas)",https://example.com/data-scientist-portfolio
7,"NLP Engineer (Python, SpaCy, Hugging Face)",https://example.com/nlp-engineer-portfolio
8,"Big Data Engineer (Hadoop, Python, Hive)",https://example.com/big-data-engineer-portfolio
9,"Computer Vision Engineer (Python, OpenCV, PyTo...",https://example.com/cv-engineer-portfolio


In [8]:
print(df.columns)


Index(['Tech Stack', 'Links'], dtype='object')


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Tech Stack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [15]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ai-researcher-portfolio'},
  {'links': 'https://example.com/ai-engineer-portfolio'}],
 [{'links': 'https://example.com/ai-engineer-portfolio'},
  {'links': 'https://example.com/ai-researcher-portfolio'}],
 [{'links': 'https://example.com/data-scientist-portfolio'},
  {'links': 'https://example.com/ml-engineer-portfolio'}],
 [{'links': 'https://example.com/ai-researcher-portfolio'},
  {'links': 'https://example.com/data-scientist-portfolio'}],
 [{'links': 'https://example.com/python-developer-portfolio'},
  {'links': 'https://example.com/data-scientist-portfolio'}],
 [{'links': 'https://example.com/ml-engineer-portfolio'},
  {'links': 'https://example.com/cv-engineer-portfolio'}],
 [{'links': 'https://example.com/ml-engineer-portfolio'},
  {'links': 'https://example.com/ai-engineer-portfolio'}],
 [{'links': 'https://example.com/ai-researcher-portfolio'},
  {'links': 'https://example.com/ai-engineer-portfolio'}],
 [{'links': 'https://example.com/ai-resear

In [14]:
job = json_res
job['skills']

['AI/ML/Math/CS',
 'AI architectures',
 'foundation models',
 'reasoning',
 'Python',
 'PyTorch',
 'TensorFlow',
 'AI reasoning',
 'symbolic and neuro-symbolic AI methods']

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Srinathareddy Suprathickreddy, currently pursuing a Master’s in Data Analytics at Clark University. With a strong foundation in data science, machine learning, and AI, along with hands-on project experience,
        excited about the opportunity to contribute to your team at [Company Name] 
        At Clark University, I have deepened my knowledge of data analytics and honed my skills in Python, machine learning, SQL, and deep learning techniques, which align well with the demands of modern AI-driven enterprises.
        I am confident in my ability to help your team leverage data and AI technologies to foster scalability, optimize processes, reduce costs, and enhance overall efficiency. 
        Your job is to write a cold email to the hiring manager regarding the job mentioned above describing the capability of me 
        in fulfilling their needs.
        also add my linkedin [https://www.linkedin.com/in/suprathickreddy/] and github links [https://github.com/suprathickreddy] to show my profile details
        Remember you are suprathickreddy, Masters in Data Analytics at Clark University. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Computer Science AI Reasoning Foundations Intern: 2025

Dear Hiring Manager,

I am excited to apply for the Computer Science AI Reasoning Foundations Intern: 2025 position at IBM Research. As a Master's student in Data Analytics at Clark University, I am confident that my strong foundation in data science, machine learning, and AI, along with hands-on project experience, make me an ideal candidate for this role.

Throughout my academic journey, I have developed a deep understanding of AI/ML/Math/CS concepts, including AI architectures, foundation models, and reasoning. My proficiency in Python, PyTorch, and TensorFlow has enabled me to work on various projects that involve AI reasoning and symbolic and neuro-symbolic AI methods. I am eager to contribute to foundational research topics in AI and develop innovative ideas and concepts that can result in highly complex, original, and creative scientific or engineering achievements.

As a data analytics student, I h